In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from faces import load_face_dataset
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import pickle

print("Imports Successful!!")

Imports Successful!!
Imports Successful!!


In [2]:
DETECT_CONF_THRESH = 0.5
RECOG_CONF_THRESH = 0.5
image_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/datasets/caltech_faces_full"
detector_main_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/face_detector"
recognizer_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/output/caltech_full_recognizer.yml"
label_encoder_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/LBP_face_recog/output/caltech_full_label_encoder.pickle"



In [9]:
# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([detector_main_folder, "deploy.prototxt"])
weightsPath = os.path.sep.join([detector_main_folder, "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)


# load the CALTECH faces dataset
print("[INFO] loading dataset...")
start = time.time()
(faces, labels) = load_face_dataset(image_folder, net, minConfidence=DETECT_CONF_THRESH, minSamples=15)
end = time.time()
print("[INFO] Detecting faces took {:.4f} seconds".format(end - start))
print("[INFO] {} images in dataset".format(len(faces)))
# encode the string labels as integers
le = LabelEncoder()
labels = le.fit_transform(labels)
# print(labels)
# construct our training and testing split
(trainX, testX, trainY, testY) = train_test_split(faces, labels, test_size=0.2, stratify=labels, random_state=42)

# train our LBP face recognizer
print("[INFO] training face recognizer...")
recognizer = cv2.face.LBPHFaceRecognizer_create(radius=2, neighbors=8, grid_x=6, grid_y=6)
start = time.time()
recognizer.train(trainX, trainY)
end = time.time()
print("[INFO] training took {:.4f} seconds".format(end - start))

# Save trained recognizer and label encoder
print("[INFO] Saving the trained recognizer and label encoder.")
# save the actual face recognition model to disk
recognizer.save(recognizer_save_path)
# write the label encoder to disk
f = open(label_encoder_save_path, "wb")
f.write(pickle.dumps(le))
f.close()


# initialize the list of predictions and confidence scores
print("[INFO] gathering predictions...")
predictions = []
confidence = []
start = time.time()
# loop over the test data
for i in range(0, len(testX)):
    # classify the face and update the list of predictions and
    # confidence scores
    (prediction, conf) = recognizer.predict(testX[i])
    predictions.append(prediction)
    confidence.append(conf)
# measure how long making predictions took
end = time.time()
print("[INFO] inference on {} images took {:.4f} seconds".format(len(testX), end - start))
# show the classification report
print(classification_report(testY, predictions, target_names=le.classes_))

# generate a sample of testing data
idxs = np.random.choice(range(0, len(testY)), size=10, replace=False)

# loop over a sample of the testing data
for i in idxs:
    # grab the predicted name and actual name
#     print(predictions)
#     print()
    predName = le.inverse_transform([predictions[i]])[0]
    actualName = le.classes_[testY[i]]
    # grab the face image and resize it such that we can easily see
    # it on our screen
    face = np.dstack([testX[i]] * 3)
    face = imutils.resize(face, width=250)
    # draw the predicted name and actual name on the image
    cv2.putText(face, "pred: {}".format(predName), (5, 25),
        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.putText(face, "actual: {}".format(actualName), (5, 60),
        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    # display the predicted name, actual name, and confidence of the
    # prediction (i.e., chi-squared distance; the *lower* the distance
    # is the *more confident* the prediction is)
    print("[INFO] prediction: {}, actual: {}, confidence: {:.2f}".format(predName, actualName, confidence[i]))
    # display the current face to our screen
    cv2.imshow("detector", face)
    key = cv2.waitKey(0)
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue
    
cv2.destroyAllWindows()



[INFO] loading face detector model...
[INFO] loading dataset...
21 different people's faces. 
 People names: ['person_a', 'person_b', 'person_c', 'person_d', 'person_e', 'person_f', 'person_g', 'person_h', 'person_i', 'person_j', 'person_k', 'person_l', 'person_m', 'person_n', 'person_o', 'person_p', 'person_q', 'person_r', 'person_s', 'person_t', 'shubhranshu_malhotra']
Returning black and white faces and labels
[INFO] Detecting faces took 35.1895 seconds
[INFO] 448 images in dataset
[INFO] training face recognizer...
[INFO] training took 0.2521 seconds
[INFO] Saving the trained recognizer and label encoder.
[INFO] gathering predictions...
[INFO] inference on 90 images took 2.2995 seconds
                      precision    recall  f1-score   support

            person_a       0.80      1.00      0.89         4
            person_b       1.00      1.00      1.00         4
            person_c       1.00      1.00      1.00         5
            person_d       1.00      1.00      1.00 